# 1. Import libraries and dataset

## Import dataset from google drive

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1t4GfxnNQr4BDD8Ob6TpuhvkiInLn2iaW'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('FinalBalancedDataset.csv')

## Import libraries

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [21]:
df = pd.read_csv('FinalBalancedDataset.csv')

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data pre-processing

## Data information

In [4]:
df.head(5)

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56745 entries, 0 to 56744
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  56745 non-null  int64 
 1   Toxicity    56745 non-null  int64 
 2   tweet       56745 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


## Data pre-processing

Seems like this dataset doesnt need to do anything...

In [22]:
df.drop('Unnamed: 0', axis=1, inplace = True)

In [36]:
x_train, x_test, y_train, y_test = train_test_split(df.values, df.Toxicity.values, test_size = .3, 
                                                    shuffle=True, random_state = 42)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((39721, 2), (39721,), (17024, 2), (17024,))

## RNN

In [7]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out


In [ ]:
batch_size = 100
n_iters = 8000
num_epochs = int(n_iters / (len(x_train) / batch_size))

